In [ ]:
# Study kinematics of  pi0s

In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit


#infile = "../NTuples/test_truth.root"
#infile = "../NTuples/full_tpceta_sample.root"

# New Stuff
#infile = "../NTuples/tpcetanocosmic_full_sample_v2.root"
infile = "../NTuples/mc_production_with_fmatch_ntuple_more_stats.root"

In [ ]:
inFile = uproot.open(infile)

inFileROOT = ROOT.TFile.Open(infile, "READ")
#h_tot_pot = inFileROOT.Get("TotalPOT")
h_tot_pot = inFileROOT.Get("TOTPOT_Clone")
TOT_POT = h_tot_pot.GetBinContent(1)
inFileROOT.Close()
TOT_POT = f"{TOT_POT:.2e}"
print("Total POT", TOT_POT)

#slc_tree = inFile["slc_truth_tree"]
slc_tree = inFile["slc_truth_tree"]
slc_reco_tree = inFile["slc_tree"]
particle_tree1 = inFile["particle_tree1"]
particle_tree2 = inFile["particle_tree2"]
daughter_tree1 = inFile["daughter_tree1"]
daughter_tree2 = inFile["daughter_tree2"]

shw_tree1 = inFile["shower_tree1"]
shw_tree2 = inFile["shower_tree2"]

slc_df = slc_tree.arrays(slc_tree.keys(), library="pd")
slc_reco_df = slc_reco_tree.arrays(slc_reco_tree.keys(), library="pd")

particle_df1 = particle_tree1.arrays(particle_tree1.keys(), library="pd")
particle_df2 = particle_tree2.arrays(particle_tree2.keys(), library="pd")

daughter_df1 = daughter_tree1.arrays(daughter_tree1.keys(), library="pd")
daughter_df2 = daughter_tree2.arrays(daughter_tree2.keys(), library="pd")

shw_df1 = shw_tree1.arrays(shw_tree1.keys(), library="pd")
shw_df2 = shw_tree2.arrays(shw_tree2.keys(), library="pd")

slc_df[:2]

In [ ]:
shw_df1["bestmatch_G4ID"] = shw_df2["bestmatch_G4ID"]
shw_df1[:2]

In [ ]:
shw_df2[:2]

In [ ]:
def isTPC(row):
    if (-200 <= row["vtx_x"] <= 200) and (-200 <= row["vtx_y"] <= 200) and (0 <= row["vtx_z"] <= 500):
        return 1
    else:
        return 0

slc_df["isTPC"] = slc_df.apply(isTPC, axis=1)
slc_reco_df["isTPC"] = slc_reco_df.apply(isTPC, axis=1)
slc_df[:2]

In [ ]:
def map_pdg_counts_to_slices(slc_df_t, other_df, pdg, new_col):
    filtered = other_df[other_df['pdg'] == pdg]

    # Step 2: Group by run and subrun and count occurrences
    counts = filtered.groupby(['run', 'subrun', 'evt', 'slc']).size()

    # Step 3: Map the counts to the smaller DataFrame
    slc_df_t[new_col] = slc_df_t.set_index(['run', 'subrun', 'evt', 'slc']).index.map(counts).fillna(0).astype(int)


map_pdg_counts_to_slices(slc_df, particle_df1, 111, "npi0")
map_pdg_counts_to_slices(slc_df, particle_df1, 221, "neta")
map_pdg_counts_to_slices(slc_df, particle_df1, 22, "ngamma")
map_pdg_counts_to_slices(slc_df, particle_df1, 13, "nmuminus")
map_pdg_counts_to_slices(slc_df, daughter_df1, 22, "ngamma_d")
slc_df[:2]

In [ ]:
interaction_codes = {
    0: "QE",
    1: "Resonant",
    2: "DIS",
    3: "Coherent",
    4: "Coherent Elastic",
    5: "Electron scatt.",
    6: "IMDAnnihilation",
    7: r"Inverse $\beta$ decay",
    8: "Glashow resonance",
    9: "AMNuGamma",
    10: "MEC",
    11: "Diffractive",
    12: "EM",
    13: "Weak Mix"
}


basic_topology_labels = {
    0:r"RES: $\nu_{\mu}\eta$",
    1:r"QE: $\nu_{\mu}\eta$",
    2:r"DIS: $\nu_{\mu}\eta$",
    3:r"MEC: $\nu_{\mu}\eta$",
    4:r"$\bar{\nu}_{\mu}\eta$",
    5:r"$\nu_{e}\eta$"+ " or "+ r"$\bar{\nu}_{e}\eta$",
    6:r"$\nu_{\mu}\eta \rightarrow 0\pi^{0}$",
    7:"Other"

}

basic_topology_selections = {
    0:"pdg == 14.0 and mode == 1.0 and neta > 0",
    1:"pdg == 14.0 and mode == 0.0 and neta > 0",
    2:"pdg == 14.0 and mode == 2.0 and neta > 0",
    3:"pdg == 14.0 and mode == 10.0 and neta > 0",
    4:"pdg == -14.0 and neta > 0",
    5:"(pdg == 12.0 or pdg == -12.0) and neta > 0",
    6: "pdg == 14.0 and mode == 1.0 and npi0 == 0 and neta > 0", 
    7: "!(pdg == 14.0 and mode == 1.0 and npi0 == 0 and neta > 0)"
}


# differentiate between a few interesting eta topologies
topology_labels1 = {

    0:r"RES: $\nu_{\mu}CC\eta \rightarrow 0\pi^{0}$",
    1:r"RES: $\nu_{\mu}CC\eta \rightarrow 1\pi^{0}$",
    2:r"RES: $\nu_{\mu}CC\eta \rightarrow 2\pi^{0}$",
    3:r"RES: $\nu_{\mu}CC\eta \rightarrow 3\pi^{0}$",
    
    4:r"RES: $\nu_{\mu}NC\eta \rightarrow 0\pi^{0}$",
    5:r"RES: $\nu_{\mu}NC\eta \rightarrow 1\pi^{0}$",
    6:r"RES: $\nu_{\mu}NC\eta \rightarrow 2\pi^{0}$",
    7:r"RES: $\nu_{\mu}NC\eta \rightarrow 3\pi^{0}$",

    8:r"Not RES: $\nu_{\mu}\eta$",
    9:r"$\bar{\nu}_{\mu}\eta$",
    10:r"$\nu_{e}\eta$"+ " or "+ r"$\bar{\nu}_{e}\eta$",
    11:r"$\nu_{\mu}$ CC $1\pi^{0}$",
    12:r"$\nu_{\mu}$ NC $1\pi^{0}$",
    13:r"$\nu_{\mu}$ CC Other",
    14:r"$\nu_{\mu}$ NC Other",
    15:r"$\nu$ Other",
    16:r"Dirt $\nu$",
    17:"Cosmic",
    18:r"RES $\nu_{\mu}\eta \rightarrow > 3\pi^{0}$"
       
}

topology_selections1 = {
    
    0:"pdg == 14.0 and mode == 1.0 and iscc == 1.0 and neta > 0 and npi0 == 0 and isTPC == 1",
    1:"pdg == 14.0 and mode == 1.0 and iscc == 1.0 and neta > 0 and npi0 == 1 and isTPC == 1",
    2:"pdg == 14.0 and mode == 1.0 and iscc == 1.0 and neta > 0 and npi0 == 2 and isTPC == 1",
    3:"pdg == 14.0 and mode == 1.0 and iscc == 1.0 and neta > 0 and npi0 == 3 and isTPC == 1",
    
    4:"pdg == 14.0 and mode == 1.0 and isnc == 1.0 and neta > 0 and npi0 == 0 and isTPC == 1",
    5:"pdg == 14.0 and mode == 1.0 and isnc == 1.0 and neta > 0 and npi0 == 1 and isTPC == 1",
    6:"pdg == 14.0 and mode == 1.0 and isnc == 1.0 and neta > 0 and npi0 == 2 and isTPC == 1",
    7:"pdg == 14.0 and mode == 1.0 and isnc == 1.0 and neta > 0 and npi0 == 3 and isTPC == 1",

    8:"pdg == 14.0 and mode != 1.0 and neta > 0 and isTPC == 1",
    9:"pdg == -14.0 and neta > 0",
    10:"(pdg == 12.0 or pdg == -12.0) and neta > 0",
    11:"pdg == 14.0 and iscc == 1.0 and neta == 0 and npi0 == 1 and isTPC == 1",
    12:"pdg == 14.0 and isnc == 1.0 and neta == 0 and npi0 == 1 and isTPC == 1",
    13:"pdg == 14.0 and iscc == 1.0 and neta == 0 and npi0 != 1 and isTPC == 1",
    14:"pdg == 14.0 and isnc == 1.0 and neta == 0 and npi0 != 1 and isTPC == 1",
    15:"(pdg == 12 or pdg == -12 or pdg == -14) and neta == 0 and isTPC == 1",
    16:"(pdg == 14 or pdg == -14 or pdg == 12 or pdg == -12) and isTPC == 0",
    17:"pdg == -1",
    #18:"(pdg != 14 and pdg != -14 and pdg != 12 and pdg != -12 and pdg != -1)"
    18:"pdg == 14 and mode == 1.0 and neta > 0 and npi0 > 3 and isTPC == 1"
    
}

topology_colors1 = {
    0:"magenta", 
    1:"purple",
    2:"violet",
    3:"deeppink",
    4:"blue",
    5:"navy",
    6:"royalblue",
    7:"lavender",
    8:"indigo",
    9:"green",
    10:"lime",
    11:"cyan",
    12:"aquamarine",
    13:"red",
    14:"maroon",
    15:"orange",
    16:"coral",
    17:"gray",
    18:"red"
}



In [ ]:
for num in range(len(topology_selections1.keys())):
    #temp = slc_df.query(topology_selections[num])
    condition = slc_df.index.isin(slc_df.query(topology_selections1[num]).index)
    slc_df.loc[condition, "TOP"] = num

slc_reco_df["TOP"] = slc_df["TOP"]
slc_reco_df[:2]

In [ ]:
df_small_filtered = slc_df[['run', 'subrun', 'evt', 'slc', 'TOP']]

particle_df1 = particle_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')
daughter_df1 = daughter_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')
shw_df1 = shw_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')

particle_df1[:2]

In [ ]:
particle_df2["TOP"] = particle_df1["TOP"]
particle_df2["pdg"] = particle_df1["pdg"]
shw_df2["TOP"] = shw_df1["TOP"]
particle_df2[:2]


In [ ]:
slc_reco_df["TOP"] = slc_df["TOP"]
daughter_df2["TOP"] = daughter_df1["TOP"]

# Want to Look at Pi0s

In [ ]:
slc_df = slc_df.query("TOP == 11 or TOP == 12")
slc_reco_df = slc_df.query("TOP == 11 or TOP == 12")
particle_df1 = particle_df1.query("TOP == 11 or TOP == 12")
particle_df2 = particle_df2.query("TOP == 11 or TOP == 12")
daughter_df1 = daughter_df1.query("TOP == 11 or TOP == 12")
daughter_df2 = daughter_df2.query("TOP == 11 or TOP == 12")
shw_df1 = shw_df1.query("TOP == 11 or TOP == 12")
shw_df2 = shw_df2.query("TOP == 11 or TOP == 12")

slc_df[:2]

In [ ]:
daughter_df2["pdg"] = daughter_df1["pdg"]

In [ ]:
# Get just the daughter photons
daughter_df1 = daughter_df1.query("pdg == 22")
daughter_df2 = daughter_df2.query("pdg == 22")

daughter_df1[:2]

In [ ]:
daughter_df2[:2]

In [ ]:
plt.hist(slc_df["ngamma_d"].values, bins=np.linspace(-0.5, 20.5, 22))
plt.show()

In [ ]:
plt.hist(slc_df["ngamma"].values, bins=np.linspace(-0.5, 20.5, 22))
plt.show()

In [ ]:
# Need at least 2 daughter gammas

slc_df = slc_df.query("ngamma_d > 1")
slc_df[:2]

In [ ]:
def get_pi0_ID(row):
    r, sr, e, s = row["run"], row["subrun"], row["evt"], row["slc"]
    m1 = (particle_df1["run"].values == r)
    m2 = (particle_df1["subrun"].values == sr)
    m3 = (particle_df1["evt"].values == e)
    m4 = (particle_df1["slc"].values == s)
    m5 = (particle_df1["pdg"].values == 111)
    m = m1 & m2 & m3 & m4 & m5
    pi0ID = int(np.array(particle_df1["G4ID"].values)[m][0])
    return pi0ID

slc_df["pi0ID"] = slc_df.apply(get_pi0_ID, axis=1)
slc_df[:2]

In [ ]:
def get_num_pi0_gammas(row):
    r, sr, e, s = row["run"], row["subrun"], row["evt"], row["slc"]
    pi0ID = row["pi0ID"]

    m1 = (daughter_df1["run"].values == r)
    m2 = (daughter_df1["subrun"].values == sr)
    m3 = (daughter_df1["evt"].values == e)
    m4 = (daughter_df1["slc"].values == s)
    m5 = (daughter_df1["parent"].values == pi0ID) 
    m = m1 & m2 & m3 & m4 & m5
    
    px = daughter_df1["px"].values[m]
    return len(px)

slc_df["npi0_gammas"] = slc_df.apply(get_num_pi0_gammas, axis=1)
slc_df[:2]

In [ ]:
plt.hist(slc_df["npi0_gammas"].values, bins=np.linspace(-0.5, 4.5, 6))
plt.show()

In [ ]:
slc_df = slc_df.query("npi0_gammas == 2")
slc_df[:2]

In [ ]:
def get_Egg(row):
    r, sr, e, s = row["run"], row["subrun"], row["evt"], row["slc"]
    pi0ID = row["pi0ID"]

    m1 = (daughter_df1["run"].values == r)
    m2 = (daughter_df1["subrun"].values == sr)
    m3 = (daughter_df1["evt"].values == e)
    m4 = (daughter_df1["slc"].values == s)
    m5 = (daughter_df1["parent"].values == pi0ID) 
    m = m1 & m2 & m3 & m4 & m5
    
    genE = daughter_df1["genE"].values[m]
    E = genE[0]*genE[1]
    return E

slc_df["Egg"] = slc_df.apply(get_Egg, axis=1)
slc_df[:2]

In [ ]:
def get_gammaID(row):
    r, sr, e, s = row["run"], row["subrun"], row["evt"], row["slc"]
    pi0ID = row["pi0ID"]

    m1 = (daughter_df1["run"].values == r)
    m2 = (daughter_df1["subrun"].values == sr)
    m3 = (daughter_df1["evt"].values == e)
    m4 = (daughter_df1["slc"].values == s)
    m5 = (daughter_df1["parent"].values == pi0ID) 
    m = m1 & m2 & m3 & m4 & m5
    
    ids = daughter_df1["G4ID"].values[m]
  
    return ids

slc_df["gammaIDs"] = slc_df.apply(get_gammaID, axis=1)
slc_df[:2]

In [ ]:
def get_costheta(row):
    r, sr, e, s = row["run"], row["subrun"], row["evt"], row["slc"]
    pi0ID = row["pi0ID"]

    m1 = (daughter_df1["run"].values == r)
    m2 = (daughter_df1["subrun"].values == sr)
    m3 = (daughter_df1["evt"].values == e)
    m4 = (daughter_df1["slc"].values == s)
    m5 = (daughter_df1["parent"].values == pi0ID) 
    m = m1 & m2 & m3 & m4 & m5
    
    px = daughter_df1["px"].values[m]
    py = daughter_df1["py"].values[m]
    pz = daughter_df1["pz"].values[m]

    px1, py1, pz1 = px[0], py[0], pz[0]
    px2, py2, pz2 = px[1], py[1], pz[1]

    norm1 = (px1**2 + py1**2 + pz1**2)**0.5
    norm2 = (px2**2 + py2**2 + pz2**2)**0.5
    dir1 = np.array([px1, py1, pz1])/norm1
    dir2 = np.array([px2, py2, pz2])/norm2

    cos = np.dot(dir1, dir2)
    return cos

slc_df["cos"] = slc_df.apply(get_costheta, axis=1)
slc_df[:2]

In [ ]:
plt.hist(slc_df["cos"].values, bins=np.linspace(-1.5, 1.5, 10))
plt.show()

In [ ]:
# reco Stuff


In [ ]:
def get_Egg_reco(row):
    r, sr, e, s = row["run"], row["subrun"], row["evt"], row["slc"]
    gIDs = row["gammaIDs"]

    m1 = (shw_df1["run"].values == r)
    m2 = (shw_df1["subrun"].values == sr)
    m3 = (shw_df1["evt"].values == e)
    m4 = (shw_df1["slc"].values == s)
    m5 = (shw_df1["bestmatch_G4ID"].values == gIDs[0])
    m6 = (shw_df1["bestmatch_G4ID"].values == gIDs[1])
    
    mp1 = m1 & m2 & m3 & m4 & m5
    mp2 = m1 & m2 & m3 & m4 & m6
    
    E1 = shw_df1["bestplane_energy"].values[mp1]
    E2 = shw_df1["bestplane_energy"].values[mp2]
    if (len(E1) == 0) or (len(E2) == 0):
        return -1
    
    E = E1[0]*E2[0]
    return E

slc_df["EggReco"] = slc_df.apply(get_Egg_reco, axis=1)
slc_df[:2]

In [ ]:
def get_costheta_reco(row):
    r, sr, e, s = row["run"], row["subrun"], row["evt"], row["slc"]
    gIDs = row["gammaIDs"]

    m1 = (shw_df1["run"].values == r)
    m2 = (shw_df1["subrun"].values == sr)
    m3 = (shw_df1["evt"].values == e)
    m4 = (shw_df1["slc"].values == s)
    m5 = (shw_df1["bestmatch_G4ID"].values == gIDs[0])
    m6 = (shw_df1["bestmatch_G4ID"].values == gIDs[1])
    
    mp1 = m1 & m2 & m3 & m4 & m5
    mp2 = m1 & m2 & m3 & m4 & m6
    
    px1 = shw_df1["dir_x"].values[mp1][0]
    py1 = shw_df1["dir_y"].values[mp1][0]
    pz1 = shw_df1["dir_z"].values[mp1][0]
    
    px2 = shw_df1["dir_x"].values[mp2][0]
    py2 = shw_df1["dir_y"].values[mp2][0]
    pz2 = shw_df1["dir_z"].values[mp2][0]

    norm1 = (px1**2 + py1**2 + pz1**2)**0.5
    norm2 = (px2**2 + py2**2 + pz2**2)**0.5
    
    dir1 = np.array([px1, py1, pz1])/norm1
    dir2 = np.array([px2, py2, pz2])/norm2

    cos = np.dot(dir1, dir2)
    return cos

slc_df["cosReco"] = slc_df.apply(get_costheta_reco, axis=1)
slc_df[:2]

In [ ]:
jhbjhb

In [ ]:
def Egg(costheta, M):
    a = 1.0/(1 - costheta)
    return 0.5*(M**2)*a

print("made the Egg function") 

In [ ]:
Meta = 547.862 # MeV
Mpi0 = 135.0 # MeV

width_eta = 1.3/1000
width_pi0 = 8.0/1000000



costheta_values = np.linspace(-1, 0.95, 1000)

yeta_c = Egg(costheta_values, Meta)
ypi0_c = Egg(costheta_values, Mpi0)

yeta_plus = Egg(costheta_values, Meta + 3*width_eta)
yeta_minus = Egg(costheta_values, Meta - 3*width_eta)

ypi0_plus = Egg(costheta_values, Mpi0 + 3*width_pi0)
ypi0_minus = Egg(costheta_values, Mpi0 - 3*width_pi0)

y400 = Egg(costheta_values, 400)
y250 = Egg(costheta_values, 250)

plt.plot(costheta_values, yeta_c, c="r", label=r"$\eta$ Peak Mass")
plt.plot(costheta_values, yeta_plus, linestyle="--", c="r")
plt.plot(costheta_values, yeta_minus, linestyle="--", c="r")
plt.plot(costheta_values, y400, linestyle="--", c="g", label="M = 400 MeV")
plt.plot(costheta_values, y250, linestyle="--", c="orange", label="M = 250 MeV")

plt.plot(costheta_values, ypi0_c, c="b", label=r"$\pi^{0}$ Peak Mass")
plt.plot(costheta_values, ypi0_plus, linestyle="--", c="b")
plt.plot(costheta_values, ypi0_minus, linestyle="--", c="b")

plt.yscale("log")
plt.xlabel(r"cos$\theta$", fontsize=14)
plt.ylabel(r"$E_{1}E_{2} [MeV^2]$", fontsize=14)
plt.legend()
plt.show()

In [ ]:
y = slc_df["Egg"].values*(1000**2)

print(min(y))
plt.hist(y, bins=50)
plt.show()

In [ ]:
from matplotlib import cm
x = slc_df["cos"].values
y = slc_df["Egg"].values*(1000**2)

plt.title("True Interactions", fontsize=20)
cmap = plt.colormaps['rainbow'].copy()  # Copy the colormap to modify it
cmap.set_under(color='white')  
hist = plt.hist2d(x, y, bins=50, density=False, cmap=cmap, vmin=0.001)
plt.scatter([], [], color='white', label=topology_labels1[11] + " or " + topology_labels1[12])
#plt.plot([0, 0.7], [0, 0.7], linestyle="--", c="r", linewidth=2)
plt.plot(costheta_values, yeta_c, c="r", label=r"$\eta$ Peak Mass")
plt.plot(costheta_values, yeta_plus, linestyle="--", c="r")
plt.plot(costheta_values, yeta_minus, linestyle="--", c="r")
plt.plot(costheta_values, y400, linestyle="--", c="g", label="M = 400 MeV")
plt.plot(costheta_values, y250, linestyle="--", c="orange", label="M = 250 MeV")

plt.plot(costheta_values, ypi0_c, c="b", label=r"$\pi^{0}$ Peak Mass")
plt.plot(costheta_values, ypi0_plus, linestyle="--", c="b")
plt.plot(costheta_values, ypi0_minus, linestyle="--", c="b")

plt.colorbar(hist[3], label='Counts')
plt.yscale("log")
plt.xlabel(r"cos$\theta$", fontsize=14)
plt.ylabel(r"$E_{1}E_{2} [MeV^2]$", fontsize=14)
plt.legend(loc ="upper left", frameon=False)
plt.show()

In [ ]:
from matplotlib import cm
x = slc_df["cos"].values
y = slc_df["Egg"].values*(1000**2)

plt.title("True Interactions", fontsize=20)
cmap = plt.colormaps['rainbow'].copy()  # Copy the colormap to modify it
cmap.set_under(color='white')  
hist = plt.hist2d(x, y, bins=1000, density=False, cmap=cmap, vmin=0.001)
plt.scatter([], [], color='white', label=topology_labels1[11] + " or " + topology_labels1[12])
#plt.plot([0, 0.7], [0, 0.7], linestyle="--", c="r", linewidth=2)
plt.plot(costheta_values, yeta_c, c="r", label=r"$\eta$ Peak Mass")
plt.plot(costheta_values, yeta_plus, linestyle="--", c="r")
plt.plot(costheta_values, yeta_minus, linestyle="--", c="r")
plt.plot(costheta_values, y400, linestyle="--", c="g", label="M = 400 MeV")
plt.plot(costheta_values, y250, linestyle="--", c="orange", label="M = 250 MeV")

plt.plot(costheta_values, ypi0_c, c="b", label=r"$\pi^{0}$ Peak Mass")
plt.plot(costheta_values, ypi0_plus, linestyle="--", c="b")
plt.plot(costheta_values, ypi0_minus, linestyle="--", c="b")

plt.colorbar(hist[3], label='Counts')
plt.yscale("log")
plt.xlabel(r"cos$\theta$", fontsize=14)
plt.ylabel(r"$E_{1}E_{2} [MeV^2]$", fontsize=14)
plt.legend(loc ="upper left", frameon=False)
plt.show()

In [ ]:
from matplotlib import cm
x = slc_df["cos"].values
y = slc_df["Egg"].values*(1000**2)

plt.title("True Interactions", fontsize=20)
#cmap = plt.colormaps['rainbow'].copy()  # Copy the colormap to modify it
#cmap.set_under(color='white')  
#hist = plt.hist2d(x, y, bins=1000, density=False, cmap=cmap, vmin=0.001)
plt.scatter(x, y, color='magenta', label=topology_labels1[11] + " or " + topology_labels1[12])
#plt.plot([0, 0.7], [0, 0.7], linestyle="--", c="r", linewidth=2)
plt.plot(costheta_values, yeta_c, c="r", label=r"$\eta$ Peak Mass")
plt.plot(costheta_values, yeta_plus, linestyle="--", c="r")
plt.plot(costheta_values, yeta_minus, linestyle="--", c="r")
plt.plot(costheta_values, y400, linestyle="--", c="g", label="M = 400 MeV")
plt.plot(costheta_values, y250, linestyle="--", c="orange", label="M = 250 MeV")

plt.plot(costheta_values, ypi0_c, c="b", label=r"$\pi^{0}$ Peak Mass")
plt.plot(costheta_values, ypi0_plus, linestyle="--", c="b")
plt.plot(costheta_values, ypi0_minus, linestyle="--", c="b")

plt.colorbar(hist[3], label='Counts')
plt.yscale("log")
plt.xlabel(r"cos$\theta$", fontsize=14)
plt.ylabel(r"$E_{1}E_{2} [MeV^2]$", fontsize=14)
plt.legend(loc ="upper left", frameon=False)
plt.show()

In [ ]:
plt.title("True Interactions", fontsize=20)
cmap = plt.colormaps['rainbow'].copy()  # Copy the colormap to modify it
cmap.set_under(color='white')  
hist = plt.hist2d(x, y, bins=50, density=False, cmap=cmap, vmin=0.001)
plt.scatter([], [], color='white', label=topology_labels1[11] + " or " + topology_labels1[12])

plt.plot(costheta_values, ypi0_c, c="b", label=r"$\pi^{0}$ Peak Mass")
plt.colorbar(hist[3], label='Counts')
#plt.yscale("log")
plt.xlabel(r"cos$\theta$", fontsize=14)
plt.ylabel(r"$E_{1}E_{2} [MeV^2]$", fontsize=14)
plt.legend(loc ="upper left", frameon=False)
plt.show()

In [ ]:
from matplotlib import cm
x = slc_df["cos"].values
y = slc_df["Egg"].values*(1000**2)

plt.title("True Interactions", fontsize=20)
#cmap = plt.colormaps['rainbow'].copy()  # Copy the colormap to modify it
#cmap.set_under(color='white')  
#hist = plt.hist2d(x, y, bins=1000, density=False, cmap=cmap, vmin=0.001)
plt.scatter(x, y, color='magenta', label=topology_labels1[11] + " or " + topology_labels1[12])
#plt.plot([0, 0.7], [0, 0.7], linestyle="--", c="r", linewidth=2)
plt.plot(costheta_values, yeta_c, c="r", label=r"$\eta$ Peak Mass")
plt.plot(costheta_values, yeta_c*(1.2**2), linestyle="--", c="r")
plt.plot(costheta_values, yeta_c*(0.8**2), linestyle="--", c="r")
plt.plot(costheta_values, y400, linestyle="--", c="g", label="M = 400 MeV")
plt.plot(costheta_values, y250, linestyle="--", c="orange", label="M = 250 MeV")

plt.plot(costheta_values, ypi0_c, c="b", label=r"$\pi^{0}$ Peak Mass")
plt.plot(costheta_values, ypi0_c*(1.2**2), linestyle="--", c="b")
#plt.plot(costheta_values, ypi0_minus, linestyle="--", c="b")

plt.colorbar(hist[3], label='Counts')
plt.yscale("log")
plt.xlabel(r"cos$\theta$", fontsize=14)
plt.ylabel(r"$E_{1}E_{2} [MeV^2]$", fontsize=14)
plt.legend(loc ="upper left", frameon=False)
plt.show()